In [1]:
%env JAVA_HOME=/usr/lib/jvm/java-21-openjdk-amd64/bin/java
%env JVM_PATH=/usr/lib/jvm/java-21-openjdk-amd64/lib/server/libjvm.so

env: JAVA_HOME=/usr/lib/jvm/java-21-openjdk-amd64/bin/java
env: JVM_PATH=/usr/lib/jvm/java-21-openjdk-amd64/lib/server/libjvm.so


## Generate Pseudo Dcos

In [44]:
import requests
# from pyserini.search import get_topics, get_qrels
from tqdm import tqdm
import json

url = "http://localhost:8000/generate"
topic_name = "dl19-passage"

# topics = get_topics(topic_name)
with open("dl19-passage_topic_info.json", "r") as f:
    topics = json.load(f)

gen_config = {
    "max_new_tokens": 128,
    "do_sample": False
}

gen_pseudo_docs = {"gen_config": gen_config}

for topic in tqdm(topics[:1], desc=f"Generating pseudo-docs for {topic_name} with probing probability included"):
    query = topic["query"]
    prompt = f"""Please write a passage to answer the question
Question: {query}
Passage:"""

    response = requests.post(url, json={"message": prompt, "gen_config": gen_config}).json()
    
    ans = [] # candidate passages
    for res in response:
        content = res["generated_text"][-1]["content"] # -1 indicates the last entry in generated_text, which is the assistant's response
        ans.append(content)
    topic["generated_passages"] = ans


gen_pseudo_docs["topics"] = topics
with open(f"{topic_name}_pseudo_docs_8rep.json", "w") as f:
    json.dump(gen_pseudo_docs, f)

{'generated_text': "Please write a passage to answer the question\nQuestion: how long is life cycle of flea\nPassage: Fleas are ectoparasites that feed on the blood of mammals and birds. The life cycle of a flea consists of four stages: egg, larva, pupa, and adult. The female flea lays her eggs on the host animal, which hatch into larvae after 2-3 weeks. The larvae feed on organic matter and debris, and then spin a cocoon to enter the pupal stage. The pupal stage lasts for 2-4 weeks, during which the flea develops into an adult. The adult flea emerges from the cocoon and begins to feed on the host animal's blood. The entire life cycle of", 'word_predictions': [{'word': '<|begin_of_text|>Please', 'predicted_probability': 0.0012386540183797479, 'predicted_label': 0}, {'word': 'write', 'predicted_probability': 0.031841836869716644, 'predicted_label': 0}, {'word': 'a', 'predicted_probability': 0.05101354047656059, 'predicted_label': 0}, {'word': 'passage', 'predicted_probability': 0.103444

In [2]:
response['generated_text']

"Please write a passage to answer the question\nQuestion: how long is life cycle of flea\nPassage: Fleas are ectoparasites that feed on the blood of mammals and birds. The life cycle of a flea consists of four stages: egg, larva, pupa, and adult. The female flea lays her eggs on the host animal, which hatch into larvae after 2-3 weeks. The larvae feed on organic matter and debris, and then spin a cocoon to enter the pupal stage. The pupal stage lasts for 2-4 weeks, during which the flea develops into an adult. The adult flea emerges from the cocoon and begins to feed on the host animal's blood. The entire life cycle of"

In [5]:
from pyserini.search import get_topics, get_qrels

topics = get_topics("msmarco-passage-dev-subset")
qrels = get_qrels("msmarco-passage-dev-subset")

In [7]:
print(len(topics))
print(len(qrels))

6980
6980


In [27]:
for topic in topics:
    print(topic, topics[topic])
    break

1102330 {'title': 'why do people grind teeth in sleep'}


In [29]:
for qid in qrels:
    print(qid, qrels[qid])
    print(1102330, qrels[1102330])
    break

300674 {7067032: '1'}
1102330 {7867438: '1'}


In [24]:
relevant_doc_cnt = []
for qid in qrels:
    relevant_doc_cnt.append(len(qrels[qid].keys()))

# descriptive of relevant documents
print(f"max relevant docs: {max(relevant_doc_cnt)}")


max relevant docs: 4


Fetching Document Content using Document ID (With Pyserini)

https://github.com/castorini/pyserini/blob/master/docs/usage-fetch.md

In [30]:
from pyserini.search.lucene import LuceneSearcher
searcher = LuceneSearcher.from_prebuilt_index('msmarco-v1-passage')
doc = searcher.doc('7067032')

import json
json_doc = json.loads(doc.raw())

json_doc['contents']

'http://en.wikipedia.org/wiki/William_Bradford_(Plymouth_Colony_governor) William Bradford (c.1590 â\x80\x93 1657) was an English Separatist leader in Leiden, Holland and in Plymouth Colony was a signatory to the Mayflower Compact. He served as Plymouth Colony Governor five times covering about thirty years between 1621 and 1657.'

In [41]:
from pyserini.search import get_topics, get_qrels
from pyserini.search.lucene import LuceneSearcher
import json
    
collection = "msmarco-passage-dev-subset"

searcher = LuceneSearcher.from_prebuilt_index('msmarco-v1-passage')

topics = get_topics(collection)
qrels = get_qrels(collection)

q_and_its_rel_docs = {}

for qid in topics:
    query = topics[qid]['title']
    q_and_its_rel_docs[qid] = {}
    q_and_its_rel_docs[qid]['qid'] = qid
    q_and_its_rel_docs[qid]['query'] = query
    q_and_its_rel_docs[qid]['rel_docs'] = []

    rel_docs = qrels[qid]
    for docid in rel_docs:
        rank = rel_docs[docid]
        doc = searcher.doc(str(docid)) # str means using external collection id
        json_doc = json.loads(doc.raw())
        content = json_doc['contents']
        q_and_its_rel_docs[qid]['rel_docs'].append({
            "docid": docid,
            "content": content,
            "rank": rank
        })

with open(f"{collection}_q_and_its_rel_docs.json", "w") as f:
    json.dump(q_and_its_rel_docs, f)

In [39]:
collection = "dl19-passage"

searcher = LuceneSearcher.from_prebuilt_index('msmarco-v1-passage')

topics = get_topics(collection)
qrels = get_qrels(collection)

print(list(qrels.items())[0])

(19335, {1017759: '0', 1082489: '0', 109063: '0', 1160863: '0', 1160871: '0', 1189088: '0', 1203500: '0', 1231806: '0', 1231807: '0', 1274615: '0', 1274620: '0', 1324075: '0', 1509459: '0', 1555317: '0', 1568085: '0', 161603: '0', 1705525: '0', 1720387: '0', 1720388: '0', 1720389: '1', 1720393: '0', 1720395: '1', 1722: '0', 1725697: '0', 1726: '0', 1729: '2', 1730: '0', 1796642: '0', 1796647: '0', 1825416: '0', 1825418: '0', 1837110: '0', 1871222: '0', 1908804: '0', 1956669: '0', 1958100: '0', 1958102: '0', 1958103: '0', 1959553: '0', 2004186: '0', 2046505: '1', 2071723: '0', 2130187: '0', 2186129: '0', 2304004: '0', 2304005: '0', 2324839: '0', 2325143: '0', 2382766: '0', 2394677: '0', 256744: '0', 256746: '0', 256750: '0', 2594897: '0', 2604487: '0', 2725017: '0', 2874503: '0', 2943092: '0', 2978577: '0', 3045565: '1', 3045567: '1', 3137952: '0', 3175481: '3', 3175483: '0', 3175484: '3', 3175485: '0', 3212083: '0', 3260688: '0', 3358752: '0', 342431: '0', 342432: '0', 3424644: '0', 34